In [1]:
    import abc, fnmatch, io, dataclasses, IPython, jinja2, inspect, collections, mistune, vdom.core, toolz.curried as toolz, base64, vdom.svg as vdom, string, traitlets, mistletoe
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    try: 
        from .html_to_vdom import html_to_vdom
        from .weave import markdown_to_vdom, markdown_to_vdom_flex
        from .utils import format_images
    except: 
        from weave import markdown_to_vdom, markdown_to_vdom_flex
        from html_to_vdom import html_to_vdom
        from utils import format_images
    ip = IPython.get_ipython()

In [2]:
def graphviz(ip, str): 
    import graphviz, vdom
    if '->' in str and not str.startswith('di'): str = 'di' + str
    return html_to_vdom()(graphviz.Source(str)._repr_svg_(), vdom.span)

In [3]:
def eval_shorthand_ipython(ip, str):
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [4]:
    def flatten(str): return ''.join(str.splitlines())

In [5]:
def embed(ip, str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return vdom.img(src=str)
        return vdom.iframe(src=str, style=dict(width='100%', height="400px"))

In [6]:
    mimes = toolz.pipe("""
        50 text/html
        70 text/latex
        80 image/svg+xml
        90 image/bmp image/png image/jpeg image/gif'
        60 text/markdown
        110 text/javascript application/javascript 
        120 text/plain
        """, str.strip, str.splitlines, toolz.map(toolz.compose(toolz.juxt(toolz.compose(int, next), list), iter, str.split)), list)

In [7]:
  def ipython_formatter(object):
        bundle, metadata = IPython.get_ipython().display_formatter.format(object)
        for type in toolz.pipe(mimes, toolz.map(toolz.second), toolz.concat):
            if type == 'text/plain': return object
            if type in bundle:  
                object = bundle[type]
                if type.startswith('image') and 'svg' not in type: 
                    object = format_images(type, bundle)._repr_html_()
                if type == 'text/html':
                    object = flatten(object)
                break
        return object

In [8]:
class StringFormatter(IPython.core.formatters.MimeBundleFormatter):
    mimebundle_formatter = traitlets.Instance(IPython.core.formatters.MimeBundleFormatter)
    formatters = traitlets.Dict()
    environment = traitlets.Instance(jinja2.Environment, kw={
        'finalize': ipython_formatter,
        'lstrip_blocks': True
    })
    @traitlets.default('formatters')
    def _default_formatters(self):return {
        'https://*' : embed,
        'http://*' : embed,
        'graph {*}': graphviz, 
        'digraph {*}': graphviz,
        ';[! ]*': eval_shorthand_ipython,
        ',[! ]*': eval_shorthand_ipython,
    }

    
    def __call__(self, object, include=None, exclude=None):
        if isinstance(object, str):
            if all(map(object.__contains__, (
                self.environment.variable_start_string, self.environment.variable_end_string))):
                object = self.environment.from_string(object).render(**self.parent.user_ns)
            for type, callable in self.formatters.items():
                if fnmatch.fnmatch(object, type):
                    result = callable(self.parent, object)
                    if result: return ip.display_formatter.format(result)
            
            else: object = markdown_to_vdom_flex()(object)
        object = self.mimebundle_formatter(object, include, exclude)
                
        return object

IPython.core.formatters.FormatterABC.register(StringFormatter);

In [9]:
def format_module_doc(type):
    if type.__doc__:
        return IPython.get_ipython().display_formatter.format(type.__doc__)

In [10]:
def attach_repr_html(vdom):
    data= vdom._repr_mimebundle_([],[])
    return {**data, 'text/html': vdom._repr_html_()}, {}

In [11]:
   def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(object.figure).print_png(bytes)
        bundle = {
            'image/png': bytes.getvalue()
        }
        return attach_repr_html(format_images('image/png', bundle))

In [12]:
def load_ipython_extension(ip): 
    ip = ip or IPython.get_ipython()
    import matplotlib.figure

    ip.display_formatter.mimebundle_formatter.for_type(matplotlib.figure.Axes, show_axes)
    
    ip.display_formatter.mimebundle_formatter = StringFormatter(
        mimebundle_formatter=ip.display_formatter.mimebundle_formatter, parent=ip)
    ip.display_formatter.mimebundle_formatter.for_type(type, format_module_doc)
    ip.display_formatter.mimebundle_formatter.for_type(__import__('vdom.core').core.VDOM, attach_repr_html)
    
def unload_ipython_extension(ip=None):
    ip.display_formatter.mimebundle_formatter = IPython.core.formatters.MimeBundleFormatter(parent=ip)
    
__name__ == '__main__' and load_ipython_extension(get_ipython())

In [13]:
("""# Testing

This is not working  {{10}}

-   {{'graph {A}'}}
-  {{"graph {B}"}}

""")

'# Testing\n\nThis is not working  {{10}}\n\n-   {{\'graph {A}\'}}\n-  {{"graph {B}"}}\n\n'